In [ ]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)

# A. Mendapatkan Informasi Dasar Data
    1. load data
    2. load data menjadi dataframe
    3. Cek dataframe
        a. Shape
        b. Head
        c. Tail
        d. Info
        e. Duplikasi (1)
        f. Duplikasi (2)

In [ ]:
# 1. load data
train = 'data/train.csv'
test = 'data/test.csv'
desc = 'data/data_description.txt'

In [ ]:
# 2. load data as dataframe
df = pd.read_csv(train)

In [ ]:
# 3. checking df
# a. shape
df.shape
print(f"Jumlah baris {df.shape[0]}")
print(f"Jumlah kolom {df.shape[1]}")

In [ ]:
# b. head
df.head(10)

In [ ]:
# c. tail
df.tail(10)

In [ ]:
# d. info
df.info()

In [ ]:
# e. cek duplikasi (1)
df.duplicated()

In [ ]:
# e. cek duplikasi (2)
df[df.duplicated()]

# B. Identifikasi Missing Value
    1. Menampilkan Missing Value
        a. Cek
    2. Membuat Fungsi Pengecekan Missing Values


In [ ]:
# 1. Menampilkan Jumlah Record Missing Value di sebuah kolom
# a. Cek Missing
df.isna().sum()

In [ ]:
# kemudian di re-index
df.isna().sum().reset_index()

In [ ]:
# 2. Membuat fungsi missing untuk mensortir missing values sebuah kolom agar mudah dibaca

def cek_missing(df):
    sum_nan = df.isna().sum().reset_index()
    # rename nama kolom
    sum_nan.columns = ['nama_Kolom', 'jumlah_Missing']
    # membuat kolom persentase
    sum_nan["persentase"] = sum_nan.jumlah_Missing/len(df)
    return sum_nan.sort_values("persentase", ascending = False)
missing = cek_missing(df)

In [ ]:
# Eksekusi fungsi cek_missing dan melihat hasilnya
missing

# C. Fixing Null Values
        1. Menyeleksi Missing Value dengan persentase kurang dari 70%
        2. Mendeskripsikan dataset ( mean, median, modus dsb)

In [ ]:
# 1. Menyeleksi Missing Value dengan persentase kurang dari 70%
miss = missing[missing.persentase < 0.7]
miss


In [ ]:
# Mencari kolom yang lebih dari 0
to_fix_col = miss[miss.persentase > 0 ].nama_Kolom.tolist()
to_fix_col

In [ ]:
# 2. Drop kolom
df2 = df.drop(missing[missing.persentase > 0.7].nama_Kolom.tolist(), 1)
to_fix = df2[df2[to_fix_col].describe().columns.tolist()]
to_fix.skew()

In [ ]:
# 2. Mendeskripsikan dataset ( mean, median, modus dsb)
to_fix.describe()

# D. Memisahkan Numerik dan Kategori
    1. Set Index
    2. Mengumpulkan record numerik
    3. Mengumpulkan record kategorikal
    4. Dataframe Numerik
    5. Dataframe Kategorikal

In [ ]:
# 1. Set Index
df2 = df2.set_index('Id')

In [ ]:
# 2. Mengumpulkan kolom berisi record numerik
col_numeric = df2.describe().columns.tolist()

In [ ]:
# 3. Mengumpulkan kolom berisi record kategori
col_categoric = df2.describe(include="object").columns.tolist()

In [ ]:
# 4. Membuat dataframe numerik
df2_numeric = df2[col_numeric]
df2_numeric

In [ ]:
# 5. Membuat dataframe kategorikal
df2_categoric = df2[col_categoric]
df2_categoric

# E. Identifikasi
    1. Outliers
    2. Skewness

In [ ]:
# Cek distribusi
# function visualisasi facetgrid
import warnings


def viz(df, types):
    num = df
    f = pd.melt(num, value_vars=num)
    g = sns.FacetGrid(f, col="variable",  col_wrap=3,
                      sharex=False, sharey=False, size=5)
    g = g.map(types, "value")
    plt.show()
    return (g)


warnings.filterwarnings('ignore')
# LotFrontage with median , GarageYrBlt mean , and MasVnrArea with median
viz(to_fix, sns.distplot)


In [ ]:
viz(df2_numeric, sns.distplot)


In [ ]:
df2_numeric['PoolArea'].unique()

In [ ]:
skew_report = df2_numeric.skew().reset_index()
skew_report.columns = ['columns', 'skew_score']
skew_report.sort_values('skew_score', ascending=False)

In [ ]:
# outliers plots
viz(df2_numeric, sns.boxplot)


In [ ]:
# outliers report
def outliers(s):
    iqr = (np.quantile(s, 0.75))-(np.quantile(s, 0.25))
    upper_bound = np.quantile(s, 0.75)+(1.5*iqr)
    lower_bound = np.quantile(s, 0.25)-(1.5*iqr)
    f = []
    for i in s:
        if i > upper_bound:
            f.append(i)
        elif i < lower_bound:
            f.append(i)
    sums = len(f)
    pros = len(f)/len(s)*100
    d = {'IQR':iqr,
         'Upper Bound':upper_bound,
        'Lower Bound':lower_bound,
        'Sum outliers': sums,'percentage outliers':pros}
    d = pd.DataFrame(d.items(),columns = ['sub','values'])
    return(d)

In [ ]:
# mencari outliers details
for i in df2_numeric:
  print(i,'\n', display(outliers(df['WoodDeckSF'])))

# F. Korelasi Antar Variabel
    1. Heatmap
    2. Matrix Korelasi variabel 'SalePrice'

In [ ]:
# 1. Heatmap
korelasi_matrix = df2.corr(method='spearman')
f, ax = plt.subplots(figsize=(30,15))
sns.heatmap(korelasi_matrix, vmax=.9, mask = np.triu(np.ones_like(korelasi_matrix)), annot=True, annot_kws={'size' : 9}, square=True)

# G. Wawasan Bisnis ( Business Insight )